In [2]:
import numpy as np
import jax.numpy as jnp

In [75]:
def diff_rank_loss(phi, time, event):
    # pycox style padding
    n = phi.shape[0]
    ones = np.ones((n,1)).reshape(-1, 1)
    pad = np.zeros_like(phi[:,:1])
    phi = np.concatenate((phi, pad),axis=1)
    # todo: add softmax
    y = np.zeros_like(phi)
    bins = np.unique(time)
    idx_durations = np.digitize(time, bins)-1
    idx_durations = idx_durations.reshape(-1, 1)
    phi_cum = np.cumsum(phi, axis=1)
    np.put_along_axis(y, idx_durations, np.ones((len(idx_durations),1)),axis=1)
    r = phi_cum@y.T
    diag_r = np.diagonal(r).reshape(1, -1)
    r = ones@diag_r -r
    print(r)
    return np.sum(r.T.sum())

def deephit_loss2_pycox(time, event, phi, sigma=1.0)->float:
    r = diff_rank_loss(phi, time, event)
    l = np.exp(-r/sigma)
    loss = np.mean(l, axis=1, keepdims=True)
    # TODO: reduction function
    return loss

In [76]:
phi = np.array([
              [1.0, 2.0, 3.0],
              [3.0, 3.0, 3.0],
              [2.0, 2.0, 2.0]])
time = np.array([5, 10, 15])
event = np.array(([1, 1, 1]))
diff_rank_loss(phi, time, event)

[[ 0.  3.  0.]
 [-2.  0. -3.]
 [-1.  2.  0.]]


-1.0

In [88]:
def deriv(x):
    rows = x.shape[0]
    columns = x.shape[1]
    mat = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if i!=j:
                mat[i,j] = -len(mat[i,j:])

            if i==j:
                mat[i,j] = i
    mat = np.triu(mat)
    return mat

x = np.array([[1.0, 2.0],
              [3.0, 3.0],
              [2.0, 2.0]])

deriv(x[:,:-1])

array([[0.],
       [0.],
       [0.]])